# Edge TPU model from Pytorch Yolov5 model

The aim of this notebook is to create an Edge TPU model from a YOLOv5 model (https://github.com/ultralytics/yolov5). The utilisation of *edgetpu-compiler* package is very difficult on Windows, this is why we use a notebook on Google Colab (to use a Linux environment).

## Compile for the Edge TPU


### Convert model to edgetpu tflite

### Retrieve the model from Wandb

In this tutorial, we retrieve the model from **Weights and Biases** (because my model is saved here), but it is not an obligation and we can upload the model on Google Drive to use it in Colab.

In [ ]:
!pip install wandb

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install

In [ ]:
%cd /content

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('tcotte/Batonnets/run_w0a5400c_model:v0', type='model')
artifact_dir = artifact.download()

### Export the model

Export in Edge TPU format from ".pt" model. Precise the *model_path* and the size of the picture (*imgsz*). Don't choose a picture size too big because the process will not stand on the **edgetpu** if the size is too important  and it will run on **CPU**. In fact, it is not possible map the large size operations especially transpose and reshape operations. You can read something like this in the log file if the operation is too heavy for the TPU :
`TRANSPOSE 1 Operation is otherwise supported, but not mapped due to some unspecified limitation`

For example, I have chosen a picture size of 448*448 for a **yolov5_n** model

In [ ]:
model_path = "/content/artifacts/run_w0a5400c_model:v0/best.pt"

!python yolov5/export.py \
--weights {model_path} \
--imgsz 448 448 \
--include edgetpu

#### Import model in local

In [ ]:
from google.colab import files

files.download('/content/artifacts/run_w0a5400c_model:v0/best-int8_edgetpu.tflite')

### Recompile model

If you have not the same versions of *Edge TPU runtime* on your computer (see versions https://coral.ai/software/#edgetpu-runtime) and on Colab, you will have an issue using the model on your computer.
In documentation it is written : "A model compiled for the Edge TPU must be executed using a corresponding version of the Edge TPU runtime. If you try to run a recently compiled model on an older runtime, then you'll see an error such as this:"
```Failed precondition: Package requires runtime version ({model_version}), which is newer than this runtime version ({edgetpu_compiler_version}).```
(link https://coral.ai/docs/edgetpu/compiler/#compiler-and-runtime-versions)

To solve this issue re-compile your model to make it compatible with the older runtime version by including the --min_runtime_version flag when you run edgetpu_compiler

First download the [Edge TPU Compiler](https://coral.ai/docs/edgetpu/compiler/):

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

Then re-compile the model to avoid the error :

In [ ]:
!edgetpu_compiler --min_runtime_version 13 /content/artifacts/run_w0a5400c_model:v0/best-int8.tflite

I have version 13 on my computer, it seems that the version 14 does not work well on Windows.

That's it.

The compiled model uses the same filename but with "_edgetpu" appended at the end.
You can now run the model on your Coral in local device with acceleration on the Edge TPU.

In [ ]:
from google.colab import files

files.download('/content/best-int8_edgetpu.tflite')



To get started, try using **edgetpu-yolo/detect.py** code :

```
python detect.py \
  --model {model_name}edgetpu.tflite \
  --input {photo_path}.jpg
```